## Protocol Extraction - Dictionary for Topic Analysis

In [1]:
from __future__ import print_function

from docx import Document
from docx.shared import Inches

import re
import nltk
from nltk import tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
import spacy
import mglearn

from sklearn.feature_extraction.text import  CountVectorizer
from sklearn.decomposition import  LatentDirichletAllocation
from collections import Counter
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import random
import numpy as np

import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

## Build a consolidate pool of  protocol extractions

#### - Should this be general or by indication?
#### - will we have enough data to build dictionary by indication?
#### - Are there other options or supplements?


In [4]:
import glob, os

df = pd.DataFrame(columns = ['extract', 'studyid'])

list_of_files = glob.glob('C:/Users/AyankoyaKayode/OneDrive - PRA Health Sciences/PRA/NLP/protocols/All/Extractions/*.csv')
for file_name in list_of_files:
    dp = pd.read_csv(file_name, header = None, encoding = "ISO-8859-1", names=['extract']) 
    dp['studyid'] = os.path.basename(file_name)
    df = pd.concat([df, dp], ignore_index=True)
    print(os.path.basename(file_name))

PE1.csv
PE10.csv
PE11.csv
PE12.csv
PE13.csv
PE14.csv
PE15.csv
PE16.csv
PE17.csv
PE18.csv
PE19.csv
PE2.csv
PE21.csv
PE22.csv
PE23.csv
PE24.csv
PE25.csv
PE26.csv
PE27.csv
PE28.csv
PE3.csv
PE4.csv
PE5.csv
PE6.csv
PE7.csv
PE8.csv
PE9.csv


### Latent Dirichlet Allocation Summarization

In [11]:
n_samples = len(df)
n_features =  round(n_samples)
n_topics = len(df.groupby('studyid')['studyid'].count())
n_top_words = 20

def print_top_words(model, feature_names, n_top_words):
    list_term_temp=[]
    #list_idx=[]

    for topic_idx, topic in enumerate(model.components_):
        #list_term_temp=[]
        #print("Topic #%d:" % topic_idx)
        #list_idx.append(topic_idx)
        #print(" ".join([feature_names[i]
        #                for i in topic.argsort()[:-n_top_words - 1:-1]]))
        for i in topic.argsort()[:-n_top_words -1:-1]:
            list_term_temp.append(feature_names[i])
            
        #list_term.append(list_term_temp)
    
    #dic=pd.DataFrame({'topic_index':list_idx, 'terms':list_term})
    #print()
    return list_term_temp


In [12]:
print("There are {} items in the dictionary".format(n_features))

There are 2580 items in the dictionary


### Create dictionaries of unigrams (single words), bigrams (2 words) and trigrams (3 words)

In [13]:
whole_data_non=df.dropna()

incld_list=whole_data_non['extract'].tolist()


data_samples_incld = incld_list

#### Unigram
tf_vectorizer_incld1 = CountVectorizer(max_df=0.6, min_df=5, ngram_range=(1,1),
                                max_features=n_features,
                                stop_words='english')

tf_incld1 = tf_vectorizer_incld1.fit_transform(data_samples_incld)

#### bigrams
tf_vectorizer_incld2 = CountVectorizer(max_df=0.6, min_df=4, ngram_range=(2,2),
                                max_features=n_features,
                                stop_words='english')

tf_incld2 = tf_vectorizer_incld2.fit_transform(data_samples_incld)

#### trigram

tf_vectorizer_incld3 = CountVectorizer(max_df=0.6, min_df=4, ngram_range=(3,3),
                                max_features=n_features,
                                stop_words='english')

tf_incld3 = tf_vectorizer_incld3.fit_transform(data_samples_incld)

In [26]:
pd.DataFrame(tf_incld1.toarray(),columns=tf_vectorizer_incld1.get_feature_names()).head()

,000,022,10,100,1000,11,12,120,13,13c,...,withdraws,wk,wocbps,women,worsening,written,year,years,yes,york
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
pd.DataFrame(tf_incld2.toarray(),columns=tf_vectorizer_incld2.get_feature_names()).head()

,000 cells,000 mm3,022 assessed,10 day,10 mg,10 subject,10 subjects,100 000,100 response,11 subject,...,written informed,year screening,years accordance,years age,years older,years prior,years screening,years treatment,york heart,zileuton medication
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
pd.DataFrame(tf_incld3.toarray(),columns=tf_vectorizer_incld3.get_feature_names()).head()

,022 assessed patients,100 000 cells,100 response headache,12 lead ecg,12 lead ecgs,12 months prior,12 visit week,12 week period,120 days dose,13 16 19,...,withdrawal criteria subject,withdrawal spermicides lactational,withdrawal subject fails,women childbearing potential,written informed consent,years accordance schedule,years age older,years prior screening,york heart association,zileuton medication interfere
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))

lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)


lda_incld1 = lda.fit(tf_incld1)
lda_incld2 = lda.fit(tf_incld2)
lda_incld3 = lda.fit(tf_incld3)

tf_feature_names_incld1 = tf_vectorizer_incld1.get_feature_names()
incld_terms1 = print_top_words(lda_incld1, tf_feature_names_incld1, n_top_words)

tf_feature_names_incld2 = tf_vectorizer_incld2.get_feature_names()
incld_terms2 = print_top_words(lda_incld2, tf_feature_names_incld2, n_top_words)

tf_feature_names_incld3 = tf_vectorizer_incld3.get_feature_names()
incld_terms3 = print_top_words(lda_incld3, tf_feature_names_incld3, n_top_words)

incld_terms_list1 = list(dict(Counter(incld_terms1).most_common(100)))
incld_terms_list2 = list(dict(Counter(incld_terms2).most_common(100)))
incld_terms_list3 = list(dict(Counter(incld_terms3).most_common(100)))

unique_incld1 = list(set(incld_terms_list1))
#unique_excld = list(set(excld_terms_list) - set(incld_terms_list)) 


Fitting LDA models with tf features, n_samples=2580 and n_features=2580...


In [31]:
sorting=np.argsort(lda.components_)[:,::-1]
features=np.array(tf_vectorizer_incld1.get_feature_names())

import mglearn
dd1 = mglearn.tools.print_topics(topics=range(5), feature_names=features,sorting=sorting, topics_per_chunk=1, n_words=10)

topic 0       
--------      
diabetes      
addition      
efficacy      
75            
cycle         
approximately 
diagnosed     
platelets     
phone         
planned       


topic 1       
--------      
ecgs          
40            
cirrhosis     
central       
classification
eosinophilia  
fluid         
make          
demonstrate   
exception     


topic 2       
--------      
hads          
laboratory    
explanation   
positive      
medical       
month         
consent       
institutional 
monitored     
includes      


topic 3       
--------      
medicinal     
comply        
exploratory   
delayed       
efficacy      
currently     
porocess      
lcm           
plan          
23            


topic 4       
--------      
infectious    
manner        
low           
permanent     
lasting       
local         
intolerable   
lives         
exclude       
moderate      




In [96]:
dict(Counter(incld_terms2).most_common(100))

{'combination tsr': 3,
 'days myoclonic': 2,
 'montelukast norfloxacin': 2,
 '30 days': 2,
 'independent ethics': 2,
 '11 subject': 2,
 'inhibitor plus': 2,
 'insulin use': 2,
 'anti insulin': 2,
 'approved written': 2,
 '14 subject': 2,
 '11 subjects': 2,
 'drug level': 2,
 'inconsistencies conmed': 2,
 'following criteria': 2,
 'aes special': 2,
 'confirmed aia': 2,
 'continue study': 2,
 'dose imp': 2,
 'condition opinion': 2,
 'confirmed active': 2,
 '72 hours': 2,
 'entire study': 2,
 '400mg day': 2,
 'alkaline phosphatase': 2,
 'increase 25': 2,
 'blind maintenance': 2,
 'clinic visit': 2,
 'consecutive days': 2,
 'inclusion subjects': 2,
 'combination chemotherapy': 2,
 'criteria subjects': 1,
 '986036 administration': 1,
 '3rd degree': 1,
 'considered adverse': 1,
 'alectinib ceritinib': 1,
 'criterion 11': 1,
 'mg prednisone': 1,
 'monitor consulted': 1,
 '16 19': 1,
 'irecist icpd': 1,
 'et visit': 1,
 'cycle cycle': 1,
 'days preceding': 1,
 'conmed reporting': 1,
 'cord com

In [93]:
dict(Counter(incld_terms3).most_common(20))

{'dose study treatment': 3,
 'impact bms 986036': 2,
 'week 52 ptfu': 2,
 '52 ptfu visit': 2,
 'safety tolerability endpoint': 2,
 '13 16 19': 2,
 'secondary safety tolerability': 2,
 'seizures 28 days': 2,
 'chemotherapy treatment period': 2,
 'clinically relevant increase': 2,
 '24 hours prior': 2,
 '13c methacetin breath': 2,
 'methacetin breath test': 2,
 'safety endpoint changes': 2,
 'period video eeg': 2,
 'baseline period video': 2,
 'end maintenance period': 2,
 'endpoint proportion subjects': 2,
 'maintenance period video': 2,
 'eeg end maintenance': 2}

In [76]:
sorting=np.argsort(lda.components_)[:,::-1]
features=np.array(tf_vectorizer_incld.get_feature_names())

In [77]:
import mglearn
dd1 = mglearn.tools.print_topics(topics=range(5), feature_names=features,sorting=sorting, topics_per_chunk=1, n_words=10)


topic 0       
--------      
video eeg     
period video  
partial onset 
maintenance period
onset seizures
end maintenance
mg qd         
safety analysis
end baseline  
dose level    


topic 1       
--------      
clinically significant
study medication
12 lead       
safety tolerability
active tb     
secondary safety
immunodeficiency virus
human immunodeficiency
including limited
exclusion criterion


topic 2       
--------      
exclusion criteria
following criteria
meet following
mbt substudy  
breath test   
methacetin breath
13c methacetin
inclusion exclusion
study inclusion
addition study


topic 3       
--------      
change baseline
secondary endpoint
entry value   
week period   
run period    
12 week       
28 day        
day run       
exploratory endpoint
baseline 28   


topic 4       
--------      
alt ast       
total bilirubin
study drug    
gilbert syndrome
upper limit   
limit normal  
drug induced  
ast alp       
normal uln    
liver injury  





PDFMIner - https://www.binpress.com/manipulate-pdf-python/

BeautifulSoup - https://www.dataquest.io/blog/web-scraping-tutorial-python/

PyTextRank - https://medium.com/@aneesha/beyond-bag-of-words-using-pytextrank-to-find-phrases-and-summarize-text-f736fa3773c5

Text Summarization with NLTK in Python - https://stackabuse.com/text-summarization-with-nltk-in-python/

Text summarization in 5 steps using NLTK - https://becominghuman.ai/text-summarization-in-5-steps-using-nltk-65b21e352b65

TFIDF - https://towardsdatascience.com/tfidf-for-piece-of-text-in-python-43feccaa74f8

NLP For Topic Modeling Summarization Of Financial Documents  https://blog.usejournal.com/nlp-for-topic-modeling-summarization-of-financial-documents-10-k-q-93070db96c1d

This is a nice subject to play with LDA on! It might also be cool to see how treating individual sentences as documents could affect topics. Computationally more expensive, but it might be feasible.

https://towardsdatascience.com/basic-nlp-on-the-texts-of-harry-potter-topic-modeling-with-latent-dirichlet-allocation-f3c00f77b0f5